# Stack Overflow 2025 Developer Survey



In [1]:
import pandas as pd
import numpy as np

In [ ]:
data_full = pd.read_csv("data/survey_results_public.csv")
data_full.head()

,ResponseId,MainBranch,Age,EdLevel,Employment,EmploymentAddl,WorkExp,LearnCodeChoose,LearnCode,LearnCodeAI,...,AIAgentOrchestration,AIAgentOrchWrite,AIAgentObserveSecure,AIAgentObsWrite,AIAgentExternal,AIAgentExtWrite,AIHuman,AIOpen,ConvertedCompYearly,JobSat
0,1,I am a developer by profession,25-34 years old,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Employed,"Caring for dependents (children, elderly, etc.)",8.0,"Yes, I am not new to coding but am learning ne...",Online Courses or Certification (includes all ...,"Yes, I learned how to use AI-enabled tools for...",...,Vertex AI,NaN,NaN,NaN,ChatGPT,NaN,When I don’t trust AI’s answers,"Troubleshooting, profiling, debugging",61256.0,10.0
1,2,I am a developer by profession,25-34 years old,"Associate degree (A.A., A.S., etc.)",Employed,NaN,2.0,"Yes, I am not new to coding but am learning ne...",Online Courses or Certification (includes all ...,"Yes, I learned how to use AI-enabled tools for...",...,NaN,NaN,NaN,NaN,NaN,NaN,When I don’t trust AI’s answers;When I want to...,All skills. AI is a flop.,104413.0,9.0
2,3,I am a developer by profession,35-44 years old,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Independent contractor, freelancer, or self-em...",None of the above,10.0,"Yes, I am not new to coding but am learning ne...",Online Courses or Certification (includes all ...,"Yes, I learned how to use AI-enabled tools for...",...,NaN,NaN,NaN,NaN,ChatGPT;Claude Code;GitHub Copilot;Google Gemini,NaN,When I don’t trust AI’s answers;When I want to...,"Understand how things actually work, problem s...",53061.0,8.0
3,4,I am a developer by profession,35-44 years old,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Employed,None of the above,4.0,"Yes, I am not new to coding but am learning ne...","Other online resources (e.g. standard search, ...","Yes, I learned how to use AI-enabled tools for...",...,NaN,NaN,NaN,NaN,ChatGPT;Claude Code,NaN,When I don’t trust AI’s answers;When I want to...,NaN,36197.0,6.0
4,5,I am a developer by profession,35-44 years old,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Independent contractor, freelancer, or self-em...","Caring for dependents (children, elderly, etc.)",21.0,"No, I am not new to coding and did not learn n...",NaN,"Yes, I learned how to use AI-enabled tools for...",...,NaN,NaN,NaN,NaN,NaN,NaN,When I don’t trust AI’s answers,"critical thinking, the skill to define the tas...",60000.0,7.0


In [ ]:
for column in data_full.columns:
    print(column)

ResponseId
MainBranch
Age
EdLevel
Employment
EmploymentAddl
WorkExp
LearnCodeChoose
LearnCode
LearnCodeAI
AILearnHow
YearsCode
DevType
OrgSize
ICorPM
RemoteWork
PurchaseInfluence
TechEndorseIntro
TechEndorse_1
TechEndorse_2
TechEndorse_3
TechEndorse_4
TechEndorse_5
TechEndorse_6
TechEndorse_7
TechEndorse_8
TechEndorse_9
TechEndorse_13
TechEndorse_13_TEXT
TechOppose_1
TechOppose_2
TechOppose_3
TechOppose_5
TechOppose_7
TechOppose_9
TechOppose_11
TechOppose_13
TechOppose_16
TechOppose_15
TechOppose_15_TEXT
Industry
JobSatPoints_1
JobSatPoints_4
JobSatPoints_5
JobSatPoints_6
JobSatPoints_7
JobSatPoints_8
JobSatPoints_9
JobSatPoints_10
JobSatPoints_11
JobSatPoints_13
JobSatPoints_14
JobSatPoints_15
JobSatPoints_16
JobSatPoints_15_TEXT
AIThreat
NewRole
ToolCountWork
ToolCountPersonal
Country
Currency
CompTotal
LanguageChoice
LanguageHaveWorkedWith
LanguageWantToWorkWith
LanguageAdmired
LanguagesHaveEntry
LanguagesWantEntry
DatabaseChoice
DatabaseHaveWorkedWith
DatabaseWantToWorkWith
Databas

In [ ]:
data_full[["Country", "Currency", "CompTotal", "LanguageHaveWorkedWith", "ConvertedCompYearly"]].head()

,Country,Currency,CompTotal,LanguageHaveWorkedWith,ConvertedCompYearly
0,Ukraine,EUR European Euro,52800.0,Bash/Shell (all shells);Dart;SQL,61256.0
1,Netherlands,EUR European Euro,90000.0,Java,104413.0
2,Ukraine,UAH Ukrainian hryvnia,2214000.0,Dart;HTML/CSS;JavaScript;TypeScript,53061.0
3,Ukraine,EUR European Euro,31200.0,Java;Kotlin;SQL,36197.0
4,Ukraine,USD United States dollar,60000.0,C;C#;C++;Delphi;HTML/CSS;Java;JavaScript;Lua;P...,60000.0


In [ ]:
mapper = {"LanguageHaveWorkedWith": "languages", "ConvertedCompYearly": "salary"}
data = data_full[["LanguageHaveWorkedWith", "ConvertedCompYearly"]].rename(columns=mapper)
data.sample(10)

,languages,salary
16460,Python;SQL,NaN
31628,NaN,315000.0
30864,Java;Kotlin,13949.0
200,Bash/Shell (all shells);C#;HTML/CSS;JavaScript...,155000.0
30905,NaN,NaN
9438,Bash/Shell (all shells);Groovy;HTML/CSS;Java;J...,111594.0
30070,NaN,NaN
8799,NaN,NaN
30411,C;C++;Elixir;GDScript;Go;Java;JavaScript;Pytho...,100000.0
21924,C#;Kotlin;PowerShell;Python;SQL;Swift;TypeScript,NaN


In [ ]:
data.languages.unique()

array(['Bash/Shell (all shells);Dart;SQL', 'Java',
       'Dart;HTML/CSS;JavaScript;TypeScript', ...,
       'Ada;Assembly;Erlang;F#;Fortran;GDScript;Go;Groovy',
       'C++;Dart;GDScript;HTML/CSS;JavaScript;PHP;PowerShell;Python;R;SQL',
       'Bash/Shell (all shells);Delphi;PowerShell;Python;SQL'],
      shape=(15468,), dtype=object)

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49123 entries, 0 to 49122
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   languages  31643 non-null  object 
 1   salary     23928 non-null  float64
dtypes: float64(1), object(1)
memory usage: 767.7+ KB


In [29]:
data = data.dropna()

In [30]:
data.languages.unique()

array(['Bash/Shell (all shells);Dart;SQL', 'Java',
       'Dart;HTML/CSS;JavaScript;TypeScript', ...,
       'Bash/Shell (all shells);C#;Groovy;Java;JavaScript;Kotlin;SQL;TypeScript',
       'C;C#;C++;GDScript;Go;HTML/CSS;Java;JavaScript;Kotlin;MATLAB;PHP;PowerShell;Python;R;Ruby;Rust;SQL;Swift;TypeScript;Visual Basic (.Net)',
       'Dart;HTML/CSS;JavaScript;Kotlin;PHP;Python;Rust;Swift;TypeScript'],
      shape=(11161,), dtype=object)

In [35]:
data["languages"] = data["languages"].apply(lambda x: x.split(";"))
print(data["languages"][0])
print(type(data["languages"][0]))

['Bash/Shell (all shells)', 'Dart', 'SQL']
<class 'list'>


C:\Users\JZaldivar\AppData\Local\Temp\ipykernel_24160\3147418012.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["languages"] = data["languages"].apply(lambda x: x.split(";"))


In [36]:
data.sample(10)

,languages,salary
37701,"[C#, Go, HTML/CSS, JavaScript, PHP, SQL]",236000.0
30391,"[Bash/Shell (all shells), HTML/CSS, Java, Java...",88000.0
1363,"[Bash/Shell (all shells), JavaScript, Kotlin, ...",75000.0
19556,"[HTML/CSS, Java, JavaScript, PHP, PowerShell, ...",49886.0
40289,"[Assembly, C, C++]",132000.0
13307,"[C#, Dart, HTML/CSS, JavaScript, Python, SQL, ...",6961.0
48068,"[C++, Python]",150000.0
42960,"[Bash/Shell (all shells), HTML/CSS, JavaScript...",87011.0
48663,"[HTML/CSS, JavaScript, Ruby, SQL]",100000.0
39375,"[C#, Java, JavaScript, SQL]",40663.0


In [40]:
data.to_json("data/salaries", orient="values")

In [43]:
data_list = data.values.tolist()
data_list

[[['Bash/Shell (all shells)', 'Dart', 'SQL'], 61256.0],
 [['Java'], 104413.0],
 [['Dart', 'HTML/CSS', 'JavaScript', 'TypeScript'], 53061.0],
 [['Java', 'Kotlin', 'SQL'], 36197.0],
 [['C',
   'C#',
   'C++',
   'Delphi',
   'HTML/CSS',
   'Java',
   'JavaScript',
   'Lua',
   'PowerShell',
   'Python',
   'SQL',
   'TypeScript',
   'VBA',
   'Visual Basic (.Net)'],
  60000.0],
 [['Java', 'Scala'], 120000.0],
 [['JavaScript', 'TypeScript'], 6240.0],
 [['Bash/Shell (all shells)',
   'HTML/CSS',
   'JavaScript',
   'Python',
   'SQL',
   'TypeScript'],
  72000.0],
 [['Java', 'Python', 'Scala'], 70000.0],
 [['HTML/CSS', 'Java', 'JavaScript', 'SQL', 'TypeScript'], 65691.0],
 [['C#'], 14400.0],
 [['HTML/CSS', 'JavaScript', 'TypeScript'], 1099.0],
 [['C#', 'C++', 'JavaScript', 'Python', 'Visual Basic (.Net)'], 87500.0],
 [['Bash/Shell (all shells)', 'JavaScript', 'Perl', 'SQL', 'TypeScript'],
  108913.0],
 [['C#', 'JavaScript', 'PowerShell', 'TypeScript'], 162421.0],
 [['C#', 'Java', 'Python',

In [ ]:
# Function wrote for midterm 
def analizar(encuesta):
    salarios = {
        "solo python": [],
        "python y otros": [],
        "no python": [],
    }
    for renglon in encuesta:
        lenguajes, salario = renglon
        python = 0
        otros = 0
        for lenguaje in lenguajes:
            if "python" in lenguaje.lower():
                python += 1
            else:
                otros += 1
        if python == 0:
            salarios["no python"].append(salario)
        elif otros == 0:
            salarios["solo python"].append(salario)
        else:
            salarios["python y otros"].append(salario)
    
    promedios = {}
    for categoria in salarios:
        promedios[categoria] = sum(salarios[categoria]) / len(salarios[categoria])

    return promedios

In [46]:
summary = analizar(data_list)
summary

{'solo python': 84029.86956521739,
 'python y otros': 101058.23474885122,
 'no python': 96590.6902598097}

In [52]:
for language, salary in summary.items():
    print(f"{language:15} {salary:10,.2f}")

solo python      84,029.87
python y otros  101,058.23
no python        96,590.69


In [53]:
# ¿De cuántas formas escribieron "Python"?

python = set()
for languages, salary in data_list:
    for language in languages:
        if "python" in language.lower():
            python.add(language)
python

{'MicroPython', 'Python'}